<div align="center">
<p align="center" style="width: 100%;">
    <img src="https://raw.githubusercontent.com/vlm-run/.github/refs/heads/main/profile/assets/vlm-black.svg" alt="VLM Run Logo" width="80" style="margin-bottom: -5px; color: #2e3138; vertical-align: middle; padding-right: 5px;"><br>
</p>
<p align="center"><a href="https://docs.vlm.run"><b>Website</b></a> | <a href="https://docs.vlm.run/"><b>API Docs</b></a> | <a href="https://docs.vlm.run/blog"><b>Blog</b></a> | <a href="https://discord.gg/AMApC2UzVY"><b>Discord</b></a> | <a href="https://chat.vlm.run"><b>Chat</b></a>
</p>
</div>

# VLM Run Orion - PCB Defect Localization

This notebook demonstrates how to use [VLM Run Orion's](https://vlm.run/orion) vision capabilities to perform high-resolution visual inspection of Printed Circuit Boards (PCBs) to identify and localize manufacturing defects. This is particularly useful for quality assurance in electronics manufacturing, detecting issues such as cold solder joints, bridged traces, missing SMDs, and component misalignment.

For more details on the API, see the [Agent API docs](https://docs.vlm.run/agents/introduction).

## Prerequisites

- Python 3.10+
- VLM Run API key (get one at [app.vlm.run](https://app.vlm.run))
- VLM Run Python Client with OpenAI extra `vlmrun[openai]`


## Setup

First, install the required packages and configure the environment.


In [2]:
# Install required packages
%pip install vlmrun[openai] --upgrade --quiet
%pip install cachetools pillow requests numpy --quiet


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import getpass

VLMRUN_API_KEY = os.getenv("VLMRUN_API_KEY", None)
if VLMRUN_API_KEY is None:
    VLMRUN_API_KEY = getpass.getpass("Enter your VLM Run API key: ")


## Initialize the VLM Run Client

We use the OpenAI-compatible chat completions interface through the VLM Run SDK.


In [4]:
from vlmrun.client import VLMRun

BASE_URL = os.getenv("VLMRUN_BASE_URL", "https://agent.vlm.run/v1")
client = VLMRun(api_key=VLMRUN_API_KEY, base_url=BASE_URL)
print("VLM Run client initialized successfully!")
print(f"Base URL: {BASE_URL}")


VLM Run client initialized successfully!
Base URL: https://agent.vlm.run/v1


## Response Models

We define Pydantic models for structured outputs. The response will include a list of identified defects with their locations and descriptions.


In [5]:
from pydantic import BaseModel, Field
from typing import List, Literal


class DefectLocation(BaseModel):
    """Spatial reference and visual evidence for a defect."""
    spatial_reference: str = Field(..., description="Spatial reference using grid system (e.g., Top-Right, Center-Left) or component ID reference (e.g., 'Adjacent to R12' or 'North of U5 chip')")
    visual_evidence: str = Field(..., description="Brief description of visual indicators (e.g., 'excessive solder build-up', 'exposed copper on pad')")


class PCBDefect(BaseModel):
    """A single identified defect on the PCB."""
    defect_type: Literal["Cold Solder Joint", "Bridged Traces/Solder Bridge", "Missing SMD", "Component Misalignment"] = Field(..., description="Category of the defect")
    location: DefectLocation = Field(..., description="Spatial reference and visual evidence for this defect")
    severity: Literal["Critical", "High", "Medium", "Low"] = Field(..., description="Severity level of the defect")


class PCBInspectionResponse(BaseModel):
    """Response containing the list of identified defects and overall inspection summary."""
    defects: List[PCBDefect] = Field(..., description="List of all identified defects on the PCB")
    inspection_summary: str = Field(..., description="Overall summary of the inspection, including total defect count and general board condition")

print("Response models defined successfully!")


Response models defined successfully!


## Helper Functions

We create helper functions to simplify making chat completion requests with structured outputs.


In [6]:
import hashlib
import json
from typing import Any, Type, TypeVar

import cachetools
from vlmrun.common.image import encode_image
from PIL import Image


T = TypeVar('T', bound=BaseModel)


def custom_key(prompt: str, image_path: str | None = None, response_model: Type[T] | None = None, model: str = "vlmrun-orion-1:auto"):
    """Custom key for caching chat_completion."""
    response_key = hashlib.sha256(json.dumps(response_model.model_json_schema(), sort_keys=True).encode()).hexdigest() if response_model else ""
    image_key = hashlib.sha256(image_path.encode()).hexdigest() if image_path else ""
    return (prompt, image_key, response_key, model)


@cachetools.cached(cache=cachetools.TTLCache(maxsize=100, ttl=3600), key=custom_key)
def chat_completion(
    prompt: str,
    image_path: str | None = None,
    response_model: Type[T] | None = None,
    model: str = "vlmrun-orion-1:auto"
) -> tuple[BaseModel | str, str]:
    """
    Make a chat completion request with structured output for PCB defect localization.

    Args:
        prompt: The prompt describing the PCB inspection task
        image_path: Path to the image file containing the PCB image
        response_model: Pydantic model for structured output
        model: Model to use (default: vlmrun-orion-1:auto)

    Returns:
        Tuple of (parsed response model or text, session_id)
    """
    content = [{"type": "text", "text": prompt}]
    
    # Add image if provided
    if image_path:
        image = Image.open(image_path)
        image_data = encode_image(image, format="JPEG")
        content.append({"type": "image_url", "image_url": {"url": image_data}})

    kwargs = {
        "model": model,
        "messages": [{"role": "user", "content": content}]
    }

    if response_model:
        kwargs["response_format"] = {
            "type": "json_schema",
            "schema": response_model.model_json_schema()
        }

    response = client.agent.completions.create(**kwargs)
    response_text = response.choices[0].message.content
    session_id = response.session_id

    if response_model:
        result = response_model.model_validate_json(response_text)
        return result, session_id

    return response_text, session_id

print("Helper functions defined!")


Helper functions defined!


## PCB Defect Localization

Perform a high-resolution visual inspection of a Printed Circuit Board to identify and localize manufacturing defects. Provide the path to an image file containing the PCB image.


In [7]:
# Combined prompt for PCB defect localization
PCB_DEFECT_LOCALIZATION_PROMPT = """
Task: Act as an expert Electronic Quality Assurance (QA) Engineer. Perform a high-resolution visual inspection of the attached Printed Circuit Board (PCB) image to identify and localize manufacturing defects.

Instructions:

Defect Categorization: Systematically scan the board for the following specific issues:

Cold Solder Joints: Identify pads where the solder appears dull, grainy, or lacks a proper concave wetting fillet.

Bridged Traces/Solder Bridges: Locate areas where solder has unintentionally connected two adjacent pads or traces, creating a potential short circuit.

Missing SMDs: Identify empty landing pads where components (resistors, capacitors, ICs) are missing based on the silkscreen markings.

Component Misalignment: Note any Surface-Mount Devices (SMDs) that are skewed or "tombstoned."

Localization: For every identified defect, provide:

Spatial Reference: Use a grid system (e.g., Top-Right, Center-Left) or reference the nearest designated component ID (e.g., "Adjacent to R12" or "North of U5 chip").

Visual Evidence: Briefly describe the visual indicators (e.g., "excessive solder build-up," "exposed copper on pad").

Output Format: Present the findings in a structured list or table for easy review by a technician.
"""

print("PCB defect localization prompt prepared!")
print(f"\nPrompt length: {len(PCB_DEFECT_LOCALIZATION_PROMPT)} characters")


PCB defect localization prompt prepared!

Prompt length: 1253 characters


In [ ]:
# Example: Inspect PCB for defects
# Replace 'path/to/your/pcb_image.jpg' with the actual path to your image
image_path = "pcb.jpg"

# Perform the inspection
result, session_id = chat_completion(
    prompt=PCB_DEFECT_LOCALIZATION_PROMPT,
    image_path=image_path,
    response_model=PCBInspectionResponse,
    model="vlmrun-orion-1:auto"
)

print(">> RESPONSE")
print(result)
print(f"\n>> SESSION ID: {session_id}")

print("\n>> INSPECTION SUMMARY")
print("=" * 80)
print(result.inspection_summary)
print("=" * 80)

print(f"\n>> IDENTIFIED DEFECTS ({len(result.defects)} total)")
print("=" * 80)
if result.defects:
    for i, defect in enumerate(result.defects, 1):
        print(f"\nDefect #{i}:")
        print(f"  Type: {defect.defect_type}")
        print(f"  Severity: {defect.severity}")
        print(f"  Location: {defect.location.spatial_reference}")
        print(f"  Visual Evidence: {defect.location.visual_evidence}")
else:
    print("No defects identified. Board appears to be in good condition.")
print("=" * 80)


>> RESPONSE
defects=[PCBDefect(defect_type='Missing SMD', location=DefectLocation(spatial_reference='Bottom-Right quadrant, adjacent to component L1', visual_evidence="Exposed copper on landing pads labeled 'LC'."), severity='High'), PCBDefect(defect_type='Missing SMD', location=DefectLocation(spatial_reference='Bottom-Right quadrant, below components L1-L3', visual_evidence='Empty landing pads with exposed copper for silkscreen labels R9, R10, R11, R12, R13, and R14.'), severity='High')] inspection_summary='High-resolution inspection of the PCB (img_fb051e) revealed several unpopulated SMD footprints despite clear silkscreen markings. No evidence of cold solder joints, solder bridges, or component misalignment was found on the populated areas of the board. A total of 7 missing components were identified in the bottom-right section.'

>> SESSION ID: 3b692c9b-ca43-41a8-8a5e-ae1ea010922c

>> INSPECTION SUMMARY
High-resolution inspection of the PCB (img_fb051e) revealed several unpopulate

---

## Conclusion

This notebook demonstrated how to use **VLM Run Orion** to perform high-resolution visual inspection of Printed Circuit Boards and identify manufacturing defects.

### Key Takeaways

1. **Structured Prompts**: Well-structured prompts with clear instructions for defect categorization and localization produce better results.
2. **Session Management**: Use `session_id` to track your inspection requests.
3. **Image Input**: Provide clear, high-quality images of PCBs for best results.
4. **Structured Output**: The model returns defects with types, locations, severity levels, and visual evidence in a structured format.

### Next Steps

- Experiment with different types of PCB images and defect categories
- Try analyzing boards with varying complexity and component density
- Explore the [VLM Run Documentation](https://docs.vlm.run) for more capabilities
- Join our [Discord community](https://discord.gg/AMApC2UzVY) for support

Happy inspecting! 🔍
